# Solitaire Gameplay (Strategic Version)

Use `random.seed(1)` for win state.

In [1]:
import pprint
import random
import numpy as np

In [2]:
class Card:

    def __init__(self, suit, value):
        """Initialize the Card class. Each card belongs to a suit, has
        a value, and can be flipped or not.
        """
        self.suit = suit
        self.value = value
        self.flipped = False

    def flip(self):
        """Flip a card. If a card is currently face_down, set it to face_up,
        and vice versa.
        """
        self.flipped = not self.flipped

    def __str__(self):
        """Return the suit and value of a card, e.g. "3 ♠".
        """
        return(f"{self.value} {self.suit}")

In [3]:
class Deck:

    def __init__(self, values, suits):
        self.cards = []
        self.cache = []
        self.populate(values, suits)
        self.shuffle()

    def __str__(self):
        return ", ".join([str(card) for card in self.cards])

    def populate(self, values, suits):
        for suit in suits:
            for value in values:
                this_card = Card(suit, value)
                self.cards.append(this_card)

    def shuffle(self):
        """Reproducible results.
        To test strategy one, use seed=4.
        To test strategy two, use seed=7.
        """
        random.seed(seed)
        random.shuffle(self.cards)

    def get_first_card(self):
        if len(self.cards) > 0:
            return self.cards[0]
        else:
            return None

    def take_first_card(self, flip=True):
        if len(self.cards) > 0:
            next_card = self.cards.pop(0)
            if flip and len(self.cards) > 0:
                self.cards[0].flip()
            return next_card
        else:
            return None

    def draw_card(self):
        if len(self.cards) > 0:
            self.cards[0].flip()
            self.cards.append(self.cards.pop(0))
            self.cards[0].flip()

In [4]:
class Stack:

    def __init__(self):
        """Initialize the Stack class.
        """
        self.cards = []

    def add_card(self, Card):
        """Add a card to a stack. Inserts the card into the first position.
        """
        self.cards.insert(0, Card)

    def flip_first_card(self):
        """Flip the first card in the stack if there are cards in the stack.
        """
        if len(self.cards) > 0:
            self.cards[0].flip()

    def get_face_up_cards(self):
        """Return the cards in the stack that are currently face_up.
        """
        return [card for card in self.cards if card.flipped]

    def __str__(self):
        """Return the current stack as the number of face_down cards and then
        each face_up card by value and suit.
        """
        returned_cards = [str(card) for card in reversed(
            self.get_face_up_cards())]
        face_down_count = len(self.cards) - len(self.get_face_up_cards())
        if face_down_count > 0:
            returned_cards.insert(0, f"{face_down_count} cards face down.")
        return ", ".join(returned_cards)

In [890]:
class Game:

    values = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]

    suits = { # Keys are unicode symbols for suits.
        u'\u2660': "black", # Spade
        u'\u2661': "red", # Hearts
        u'\u2663': "black", # Clubs
        u'\u2662': "red", # Diamonds
    }

    def __init__(self):
        """Initialize the Game class.
        """
        self.moves = 0
        
        # Instantiate the Deck.
        self.deck = Deck(self.values, self.suits)

        # Instantiate the Stacks.
        self.play_stacks = []

        # For each of the seven stacks...
        for i in range(7):
            this_stack = Stack()
            # ...take the first card from the Deck without flipping it and add
            # it to the current stack by inserting it into the first position.
            # The first stack will have 1 card, the second 2...the last, 7.
            [this_stack.add_card(self.deck.take_first_card(flip=False))
             for j in range(i + 1)]
            # Flip the topmost card in the stack.
            this_stack.flip_first_card()
            # Add the populated stack to the list of playable stacks.
            self.play_stacks.append(this_stack)

        # Populate the four Ace stacks.
        self.ace_stacks = {suit: Stack() for suit in self.suits}

        # Flip the first card in the Deck.
        self.deck.cards[0].flip()

    def show_board(self):
        board = {
            "deck": str(self.deck),
            "stacks": [str(stack) for stack in self.play_stacks],
            "ace stacks": {suit: str(stack) for suit, stack in
                           self.ace_stacks.items()}
        }
        return board

    def check_card_order(self, higher, lower):
        """Determine whether or not two cards can be placed consecutively. Make
        sure that both cards are of differing suits with suits_different. Then,
        ensure that the cards are consecutive with values_consecutive by check-
        ing that the value of the higher card is exactly one more than the
        value of the lower one. If both criteria are met, then the card order
        is allowed.
        """
        suits_different = self.suits[higher.suit] != self.suits[lower.suit]
        values_consecutive = self.values[self.values.index(higher.value)
                                        - 1] == lower.value
        return suits_different and values_consecutive

    def win(self):
        """Determine when the game is won. Deck must be empty, stacks must be
        empty, and Ace stacks must be full.
        """
        deck_empty = len(self.deck.cards) == 0
        stacks_empty = all(len(stack.cards) == 0 for stack in self.play_stacks)
        aces_full = all(len(stack.cards) == 13 for suit,
                        stack in self.ace_stacks.items())
        return deck_empty and stacks_empty and aces_full

    def add_to_ace_stack(self, card):
        """See if the current card can be added to the corresponding Ace stack.
        """
        if card is None: # do nothing if there is no current card
            return False
        # If there are cards in the Ace stack...
        elif len(self.ace_stacks[card.suit].cards) > 0:
            # ...get the value of topmost card.
            highest_value = self.ace_stacks[card.suit].cards[0].value
            # Check if value of the current card is higher than the topmost.
            if self.values[self.values.index(highest_value) + 1] == card.value:
                # If so, play the card to the corresponding Ace stack.
                self.ace_stacks[card.suit].cards.insert(0, card)
                return True
        else:
            return False
    
    def play_strategy_one(self):
        """Always play an Ace or Deuce wherever you can immediately.
        """
        card_added = self.deck.get_first_card()
        
        #--- 1.1 Always play an Ace wherever you can immediately. ---#
        
        # Play an Ace from the deck to its own stack.
        if card_added is not None and card_added.value == "A":
            card_added = self.deck.take_first_card()
            self.ace_stacks[card_added.suit].cards.insert(0, card_added)
            if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from deck to Ace stack")
            return True
        
        # Play an Ace from the board to its own stack.
        for stack in self.play_stacks:
            if len(stack.cards) > 0 and stack.cards[0].value == "A":
                card_added = stack.cards.pop(0)
                self.ace_stacks[card_added.suit].cards.insert(0, card_added)
                if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                return True
        
        #--- 1.2 Always play a Deuce wherever you can immediately. ---#
        
        # Play a Deuce from the deck to an Ace stack.
        if card_added is not None and card_added.value == "2":
            if self.add_to_ace_stack(self.deck.get_first_card()):
                card_added = self.deck.take_first_card()
                if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from deck to Ace stack")
                return True

        # Play a Deuce from the board to an Ace stack.
        for stack in self.play_stacks:
            if len(stack.cards) > 0 and stack.cards[0].value == "2":
                if self.add_to_ace_stack(stack.cards[0]):
                    card_added = stack.cards.pop(0)
                    if verbose == True:
                        print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                    return True
        return False
    
    def play_strategy_two(self):
        """Always make the play or transfer that frees (or allows a play that frees) a downcard,
        regardless of any other considerations. To test this strategy, use seed=7.
        """
        # For each stack in the deck...
        for stack in self.play_stacks:
            # ...get the number of face down cards.
            stack_face_up = stack.get_face_up_cards()
            num_stack_face_down = len(stack.cards) - len(stack_face_up)

            if num_stack_face_down > 0:
                # If there's only one card on it, check if it can be played to an ace stack.
                if len(stack_face_up) == 1 and self.add_to_ace_stack(stack_face_up[0]):
                        card_added = stack.cards.pop(0)
                        if verbose == True:
                            print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                        return True
                # Else, try to move the stack somewhere else.
                else:
                    # Check all the available stacks.
                    for pile in self.play_stacks:
                        # Get all the face_up cards in the pile.
                        pile_face_up = pile.get_face_up_cards()
                        # Skip the current stack.
                        if pile is not stack and len(pile_face_up) > 0:
                            cards_to_move = stack_face_up[:len(stack_face_up)]
                            if self.check_card_order(pile.cards[0], cards_to_move[-1]):
                                if verbose == True:
                                    print("Stack:", [(card.suit, card.value) for card in stack_face_up], "\nPile:",  [(card.suit, card.value) for card in pile_face_up])
                                    print("Cards to move:", [(card.suit, card.value) for card in cards_to_move])
                                [pile.cards.insert(0, card) for card in reversed(cards_to_move)]
                                stack.cards = stack.cards[len(cards_to_move):]
                                if verbose == True:
                                    print(f"Move {self.moves}: Move {len(cards_to_move)} cards between piles")
                                return True
        return False

    def play_strategy_three(self):
        """When faced with a choice, always make the play or transfer that frees (or allows a play that frees)
        the downcard from the biggest pile of downcards.
        """
        smallest = 13
        for stack in self.play_stacks:
            stack_face_up = stack.get_face_up_cards()
            num_stack_face_down = len(stack.cards) - len(stack_face_up)
            if num_stack_face_down < smallest and num_stack_face_down != 0:
                smallest = num_stack_face_down
        return smallest
    
    def play_strategy_four(self, cards):
        """Transfer cards from column to column only to allow a downcard to be freed or to make the columns
        smoother. To test this strategy, use seed=1."""
        # Return the suits of the given cards.
        return [(card.suit, card.value) for card in cards[1::2]]
    
    def play_strategy_five(self):
        """Don't clear a spot unless there's a King IMMEDIATELY waiting to occupy it. To test this strategy,
        use seed=1.
        """
        king_waiting = self.play_strategy_six()
        
        if king_waiting:
        
            for index, stack in enumerate(self.play_stacks):
                stack_face_up = stack.get_face_up_cards()
                num_stack_face_down = len(stack.cards) - len(stack_face_up)
                # If the stack is already empty
                if num_stack_face_down == 0 and len(stack_face_up) == 0:
                    if king_waiting < 0:
                        card_added = self.deck.take_first_card()
                        stack.add_card(card_added)
                        if verbose == True:
                            print(f"Move {self.moves}: Move {str(card_added)} from deck to empty stack")
                        return True
                    else:
                        cards_to_move = self.play_stacks[king_waiting].get_face_up_cards()
                        # Move to empty stack
                        [stack.cards.insert(0, card) for card in reversed(cards_to_move)]
                        self.play_stacks[king_waiting].cards = self.play_stacks[king_waiting].cards[len(cards_to_move):]
                        if verbose == True:
                            print(f"Move {self.moves}: Move cards from board to empty stack")
                        return True
                # If there's only one card in the stack
                elif num_stack_face_down == 1 and len(stack_face_up) == 1:
                    if self.add_to_ace_stack(stack_face_up[0]):
                            card_added = stack.cards.pop(0)
                            if verbose == True:
                                print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                            return True
                # If all cards in the stack are face up
                elif num_stack_face_down == 0 and len(stack_face_up) > 0:
                    #print([(card.suit, card.value) for card in stack.cards])
                    # Clear the spot
                    options = []
                    for pile in self.play_stacks:
                        # See if the card can be transfered
                            pile_face_up = pile.get_face_up_cards()
                            # Skip the working stack
                            if pile is not stack and len(pile_face_up) > 0:
                                cards_to_move = stack_face_up[:len(stack_face_up)]
                                if self.check_card_order(pile.cards[0], cards_to_move[-1]):
                                    options.append(pile)
                                    #print("Stack:", [(card.suit, card.value) for card in stack_face_up], "\nPile:",  [(card.suit, card.value) for card in pile_face_up])
                                    #print("Cards to move:", [(card.suit, card.value) for card in cards_to_move])
                                    #smooth_partners = self.play_strategy_four(pile_face_up)
                                    #print("Smooth partners:", smooth_partners)
                    #print(options)
                    for pile in options:
                        pile_face_up = pile.get_face_up_cards()                    
                        smooth_partners = self.play_strategy_four(pile_face_up)
                        cards_to_move = stack_face_up[:len(stack_face_up)]
                        if np.all([cards_to_move[-1].suit == suit for suit in smooth_partners]):
                            #print("Chosen:", [(card.suit, card.value) for card in pile_face_up])
                            [pile.cards.insert(0, card) for card in reversed(cards_to_move)]
                            stack.cards = stack.cards[len(cards_to_move):]
                            if verbose == True:
                                print(f"Move {self.moves}: Move {len(cards_to_move)} cards between piles")
                            if king_waiting < 0:
                                card_added = self.deck.take_first_card()
                                stack.add_card(card_added)
                                if verbose == True:
                                    print(f"Move {self.moves}: Move {str(card_added)} from deck to empty stack")
                                return True
                            else:
                                cards_to_move = self.play_stacks[king_waiting].get_face_up_cards()
                                # Move to empty stack
                                [stack.cards.insert(0, card) for card in reversed(cards_to_move)]
                                self.play_stacks[king_waiting].cards = self.play_stacks[king_waiting].cards[len(cards_to_move):]
                                if verbose == True:
                                    print(f"Move {self.moves}: Move cards from board to empty stack")
                                return True
        return False
    
    def play_strategy_six(self):
        """Only play a King that will benefit the column(s) with the biggest pile of downcards, unless the
        play of another King will at least allow a transfer that frees a downcard. 
        """
        # Check the board for a King
        king_loc = 0
        best_king = 0
        
        for index, stack in enumerate(self.play_stacks):
            stack_face_up = stack.get_face_up_cards()
            num_stack_face_down = len(stack.cards) - len(stack_face_up)
            # If all cards in the stack are face up
            if num_stack_face_down > 0 and len(stack_face_up) > 0:
                # Check that there's a waiting King and choose the one with the biggest num cards face down
                if "K" in [(card.value) for card in stack_face_up] and num_stack_face_down > best_king:
                    best_king = num_stack_face_down
                    #king_waiting = stack_face_up
                    # print(index, [(card.suit, card.value) for card in king_waiting])
                    king_loc = index
        
        # Check the deck for a king
        if self.deck.get_first_card() is not None and self.deck.get_first_card().value == "K":
            king_loc = -1
        
        return king_loc
    
    def play_strategy_seven(self):
        """Only build your Ace stacks (with anything other than an Ace or Deuce) when the following apply:
            1) The move WILL NOT interfere with Next Card Protection
            2) The move WILL allow a play or transfer that frees a downcard
                - Criteria satisfied in play_strategy_two()
            3) The move WILL open up a space for a same-suit card pile transfer to free a downcard
            4) The move WILL clear a spot for an IMMEDIATE Waiting King
        To test this strategy, use seed=5.
        """        
        # 1) The move WILL NOT interfere with Next Card Protection (play from the board)
        for stack in self.play_stacks:
            # If there are cards in the stack and the topmost card can be
            # added to its corresponding Ace stack...
            if len(stack.cards) > 0 and self.next_card_protection(stack.cards[0], current_stack=stack):
                if self.add_to_ace_stack(stack.cards[0]):
                    card_added = stack.cards.pop(0)
                    if verbose == True:
                        print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                    return True

        # 1) The move WILL NOT interfere with Next Card Protection (play from the deck)
        card_to_add = self.deck.get_first_card()
        if self.next_card_protection(card_to_add):
            if self.add_to_ace_stack(card_to_add):
                card_added = self.deck.take_first_card()
                if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from deck to Ace stack")
                return True
                
    def next_card_protection(self, card_to_add, current_stack=None):
        """Wait to build the Ace stack until all of these criteria are met. Note that enabling this may
        cause previously winning hands to lose. Test this strategy with seed=5 or seed=1.
        """
        # Turn off to debug
        disable = False
        
        if card_to_add is not None:
            next_lowest = Game.values.index(card_to_add.value) - 1
        
            # Check if there's another card with the same value on the board
            one_on_board = []
            for stack in self.play_stacks:
                if current_stack and stack is not current_stack:
                    one_on_board.append([(card.suit, card.value) for card in stack.get_face_up_cards() if (card.value == card_to_add.value and Game.suits[card.suit] == Game.suits[card_to_add.suit])])
            criteria1 = any(one_on_board)

            # Check if both next lowest cards of opposite suit are on the board
            both_on_board = []
            for stack in self.play_stacks:
                if current_stack and stack is not current_stack:
                    both_on_board.append([(card.suit, card.value) for card in stack.get_face_up_cards() if (card.value == Game.values[next_lowest] and Game.suits[card.suit] != Game.suits[card_to_add.suit])])
            criteria2 = np.count_nonzero(both_on_board) == 2

            # Check if both next lowest cards of opposite suit are in ace stacks
            both_in_aces = []
            for i in list(play.ace_stacks.keys()):
                both_in_aces.append([(card.suit, card.value) for card in self.ace_stacks[i].get_face_up_cards() if card.value == Game.values[next_lowest] and Game.suits[card.suit] != Game.suits[card_to_add.suit]])
            criteria3 = np.count_nonzero(both_in_aces) == 2
        
            return any([criteria1, criteria2, criteria3, disable])
    
    def play_strategy_eight(self):
        """DON'T PLAY OR TRANSFER A 5, 6, 7, OR 8 ANYWHERE UNLESS AT LEAST ONE OF THE FOLLOWING WILL APPLY:
            1) The card will be smooth with it's next highest even/odd partner in the column
            2) The move will allow a play or transfer that will IMMEDIATELY free a downcard
                - Criteria satisfied in play_strategy_two()
            3) There have not been any other cards already played to the column
            4) You have ABSOLUTELY no other choice but to continue playing (this is not a good sign)
        Test this strategy with seed=1.
        """
        card_to_add = self.deck.get_first_card()
        
        # Wait to play 5s, 6s, 7s and 8s from deck
        if card_to_add is not None and card_to_add.value in ['5', '6', '7', '8']:
            for stack in self.play_stacks:
                # For each non-empty stack...
                if len(stack.cards) > 0:
                    stack_face_up = stack.get_face_up_cards()
                    # 1) The card will be smooth with it's next highest even/odd partner in the column
                    if len(stack_face_up) > 1:
                        smooth_partners = self.play_strategy_four(stack_face_up)
                        if card_to_add.suit == smooth_partners[0][0]:
                            if self.check_card_order(stack.cards[0], card_to_add):
                                card_added = self.deck.take_first_card()
                                stack.add_card(card_added)
                                if verbose == True:
                                    print(f"Move {self.moves}: Play {str(card_added)} from deck to board")
                                return True
                    # 3) There have not been any other cards already played to the column
                    elif len(stack_face_up) == 1:
                        # ...check that the cards are in an allowable order.
                        if self.check_card_order(stack.cards[0], card_to_add):
                            #Add the card to the stack.
                            card_added = self.deck.take_first_card()
                            stack.add_card(card_added)
                            if verbose == True:
                                print(f"Move {self.moves}: Play {str(card_added)} from deck to board")
                            return True
                    # 4) You have ABSOLUTELY no other choice to continue playing
                    else:
                        if self.check_card_order(stack.cards[0], card_to_add):
                        # Add the card to the stack.
                            card_added = self.deck.take_first_card()
                            stack.add_card(card_added)
                            if verbose == True:
                                print(f"Move {self.moves}: Play {str(card_added)} from deck to board")
                            return True
        # Otherwise, immediately play any eligible cards from the deck
        elif card_to_add is not None:
            for stack in self.play_stacks:
                # For each non-empty stack...
                if len(stack.cards) > 0:
                    # ...check that the cards are in an allowable order.
                    if self.check_card_order(stack.cards[0], card_to_add):
                        # Add the card to the stack.
                        card_added = self.deck.take_first_card()
                        stack.add_card(card_added)
                        if verbose == True:
                            print(f"Move {self.moves}: Play {str(card_added)} from deck to board")
                        return True
    
    def play_strategy_nine(self):
        """When you get to a point that you think all of your necessary cards are covered and you can't just
        get to them, IMMEDIATELY play ANY CARDS YOU CAN to their appropriate Ace stacks. You may have to
        rearrange existing piles to allow blocked cards freedom to be able to go to their Ace stack. Hopefully
        this will clear an existing pile up to the point that you can use an existing pile upcard to
        substitute for the necessary covered card.
        """
        #1: Check if there are any cards that are playable to an Ace stack.
        for stack in self.play_stacks:
            # If there are cards in the stack and the topmost card can be
            # added to its corresponding Ace stack...
            if len(stack.cards) > 0 and self.add_to_ace_stack(stack.cards[0]):
                card_added = stack.cards.pop(0)
                if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from board to Ace stack")
                return True

        #2: Check if any cards in the deck are playable to an Ace stack.
        if self.add_to_ace_stack(self.deck.get_first_card()):
            card_added = self.deck.take_first_card()
            if verbose == True:
                    print(f"Move {self.moves}: Play {str(card_added)} from deck to Ace stack")
            return True
        
        #5: Move cards around playable stacks.
        # For each stack on the board...
        for stack in self.play_stacks:
            # ...get all the face_up cards in that stack.
            stack_face_up = stack.get_face_up_cards()
            # If there are face_up cards in the stack...
            if len(stack_face_up) > 0:
                # ...check every other playable stack.
                for pile in self.play_stacks:
                    # Get all the face_up cards in the pile.
                    pile_face_up = pile.get_face_up_cards()
                    # If there are face_up cards in a different stack...
                    if pile is not stack and len(pile_face_up) > 0:
                        # ...choose how many cards to move.
                        for num_cards_to_move in range(1,len(stack_face_up) + 1):
                            cards_to_move = stack_face_up[:num_cards_to_move]
                            # Check if the card order is allowable.
                            if self.check_card_order(pile.cards[0],cards_to_move[-1]):
                                stack_face_down = len(stack.cards) - len(stack_face_up)
                                pile_face_down = len(pile.cards) - len(pile_face_up)
                                if pile_face_down < stack_face_down:
                                    [pile.cards.insert(0, card) for card in reversed(cards_to_move)]
                                    stack.cards = stack.cards[num_cards_to_move:]
                                    if verbose == True:
                                        print(f"Move {self.moves}: Move {num_cards_to_move} cards between piles")
                                    return True
                                elif stack_face_down == 0 and len(cards_to_move) == len(stack.cards):
                                    [pile.cards.insert(0, card) for card in reversed(cards_to_move)]
                                    stack.cards = []
                                    if verbose == True:
                                        print(f"Move {self.moves}: Move {num_cards_to_move} cards between piles")
                                    return True
                                else:
                                    # See if the move will free a card to an Ace stack
                                    next_up = stack_face_up[num_cards_to_move]
                                    next_lowest = Game.values.index(next_up.value) - 1
                                    result = np.count_nonzero([card for card in self.ace_stacks[next_up.suit].get_face_up_cards() if card.value == Game.values[next_lowest]])
                                    if result:
                                        [pile.cards.insert(0, card) for card in reversed(cards_to_move)]
                                        stack.cards = stack.cards[num_cards_to_move:]
                                        if verbose == True:
                                            print(f"Move {self.moves}: Move {num_cards_to_move} cards between piles")
                                        return True

            
    def take_strategic_turn(self, verbose=False):
        
        #0: Flip any face_down cards at the end of the stack.
        [stack.cards[0].flip() for stack in self.play_stacks if len(stack.cards) > 0 and not stack.cards[0].flipped]
        
        if self.play_strategy_one():
            return True
        
        if self.play_strategy_two():
            return True
        
        if self.play_strategy_five():
            return True
        
        if self.play_strategy_seven():
            return True
        
        if self.play_strategy_eight():
            return True
        
        if self.play_strategy_nine():
            return True
        
        return False

    def simulate(self, draw=False, turn="strategic", verbose=False):
        if verbose==True:
            print()
            pp.pprint(play.show_board())
            print(); print()
            
        # Clear cache if last turn wasn't a card draw.
        if not draw:
            self.deck.cache = []

        # Take a turn.
        if turn == "simple":
            turn_result = self.take_simple_turn(verbose=verbose)
        else:
            turn_result = self.take_strategic_turn(verbose=verbose)
        
        # If the turn was successful, take another.
        if turn_result:
            self.moves += 1
            self.simulate(verbose=verbose)

        else:
            self.moves += 1
            # Try to draw from deck.
            if len(self.deck.cards) > 0:
                current_card = self.deck.cards[0]

                if current_card in self.deck.cache:
                    if verbose == True:
                        print("No more moves left!")
                        print(f"Moves: {self.moves}")
                    return

                else:
                    self.deck.draw_card()
                    if verbose == True:
                        print(f"Move {self.moves}: Draw a card")
                    self.deck.cache.append(current_card)
                    return self.simulate(draw=True, verbose=verbose)

            else:
                if verbose == True:
                    print("No more moves left!")
                    print(f"Moves: {self.moves}")
                return


In [892]:
pp = pprint.PrettyPrinter(indent = 4)

seed = 1

turn = "strategic"
verbose = True

for i in range(1):
    play = Game()
    play.simulate(turn=turn, verbose = verbose)

    if(play.win()):
        result = 1
        print("You won!")
    else:
        result = 0
        print("You lost! Try again.")

    print()
    pp.pprint(play.show_board())
    print(); print()
    print(f"Seed for this run: {seed}")


{   'ace stacks': {'♠': '', '♡': '', '♢': '', '♣': ''},
    'deck': '8 ♣, 4 ♠, 6 ♡, 2 ♡, 5 ♡, 6 ♢, A ♠, 2 ♣, 4 ♢, 2 ♠, 7 ♢, 7 ♠, A ♡, '
            'Q ♡, 3 ♢, 5 ♣, 3 ♣, 6 ♣, 8 ♠, 4 ♡, 5 ♠, 10 ♢, J ♣, 9 ♠',
    'stacks': [   'J ♢',
                  '1 cards face down., Q ♣',
                  '2 cards face down., K ♣',
                  '3 cards face down., 6 ♠',
                  '4 cards face down., J ♡',
                  '5 cards face down., K ♠',
                  '6 cards face down., J ♠']}


Stack: [('♡', 'J')] 
Pile: [('♣', 'Q')]
Cards to move: [('♡', 'J')]
Move 0: Move 1 cards between piles

{   'ace stacks': {'♠': '', '♡': '', '♢': '', '♣': ''},
    'deck': '8 ♣, 4 ♠, 6 ♡, 2 ♡, 5 ♡, 6 ♢, A ♠, 2 ♣, 4 ♢, 2 ♠, 7 ♢, 7 ♠, A ♡, '
            'Q ♡, 3 ♢, 5 ♣, 3 ♣, 6 ♣, 8 ♠, 4 ♡, 5 ♠, 10 ♢, J ♣, 9 ♠',
    'stacks': [   'J ♢',
                  '1 cards face down., Q ♣, J ♡',
                  '2 cards face down., K ♣',
                  '3 cards face down., 6 ♠',
                  '4

{   'ace stacks': {   '♠': 'A ♠, 2 ♠, 3 ♠, 4 ♠, 5 ♠, 6 ♠',
                      '♡': 'A ♡, 2 ♡, 3 ♡, 4 ♡, 5 ♡, 6 ♡, 7 ♡, 8 ♡',
                      '♢': 'A ♢, 2 ♢, 3 ♢, 4 ♢, 5 ♢',
                      '♣': 'A ♣, 2 ♣, 3 ♣, 4 ♣, 5 ♣, 6 ♣'},
    'deck': '9 ♠, 6 ♢, 7 ♠',
    'stacks': [   'K ♢, Q ♣, J ♡, 10 ♣, 9 ♢, 8 ♠, 7 ♢',
                  'K ♠, Q ♡, J ♠, 10 ♢, 9 ♣, 8 ♢, 7 ♣',
                  '10 ♡',
                  'K ♡, Q ♠, J ♢, 10 ♠, 9 ♡, 8 ♣',
                  '',
                  '',
                  'K ♣, Q ♢, J ♣']}


Move 85: Play 9 ♠ from deck to board

{   'ace stacks': {   '♠': 'A ♠, 2 ♠, 3 ♠, 4 ♠, 5 ♠, 6 ♠',
                      '♡': 'A ♡, 2 ♡, 3 ♡, 4 ♡, 5 ♡, 6 ♡, 7 ♡, 8 ♡',
                      '♢': 'A ♢, 2 ♢, 3 ♢, 4 ♢, 5 ♢',
                      '♣': 'A ♣, 2 ♣, 3 ♣, 4 ♣, 5 ♣, 6 ♣'},
    'deck': '6 ♢, 7 ♠',
    'stacks': [   'K ♢, Q ♣, J ♡, 10 ♣, 9 ♢, 8 ♠, 7 ♢',
                  'K ♠, Q ♡, J ♠, 10 ♢, 9 ♣, 8 ♢, 7 ♣',
                  '10 ♡, 9 ♠',
           

In [815]:
test = [('red', 'K'), ('black', '5'), ('black', 'K'), ('black', '4'), ('red', '9'), ('red', '9'), ('red', 'Q')]

In [818]:
set(test)

{('black', '4'),
 ('black', '5'),
 ('black', 'K'),
 ('red', '9'),
 ('red', 'K'),
 ('red', 'Q')}

## Results

Simple:

* `n=500`
    - Num wins: 3
    - Win rate: 0.006
    - Avg. num moves: 60.112
* `n=1000`
    - Num wins: 10
    - Win rate: 0.01
    - Avg. num moves: 60.162

Strategic:

* `n=500`
    - Num wins: 13
    - Win rate: 0.026
    - Avg. num moves: 60.052
* `n=1000`
    - Num wins: 19
    - Win rate: 0.019
    - Avg. num moves: 60.167

## ETA: Fixed a bug!

I had a bug in `play_strategy_six()` where I was taking a card from the deck, and then in `play_strategy_five()` I again took another card from the deck. This caused the algorithm to play non-eligible cards to empty stacks. Here's the reproduced bug:

```python
    def play_strategy_five(self):
        """Don't clear a spot unless there's a King IMMEDIATELY waiting to occupy it. To test this strategy,
        use seed=1.
        """
        king_waiting = self.play_strategy_six()
        
        if king_waiting:
        
            for index, stack in enumerate(self.play_stacks):
                stack_face_up = stack.get_face_up_cards()
                num_stack_face_down = len(stack.cards) - len(stack_face_up)
                # If the stack is already empty
                if num_stack_face_down == 0 and len(stack_face_up) == 0:
                    if king_waiting < 0:
                        card_added = self.deck.take_first_card() <--- CARD IS TAKEN HERE
                        stack.add_card(card_added)
                        if verbose == True:
                            print(f"Move {self.moves}: Move {str(card_added)} from deck to empty stack")
                        return True
                    else:
                    
                       # Snip
                       
        return False
        
    def play_strategy_six(self):
        """Only play a King that will benefit the column(s) with the biggest pile of downcards, unless the
        play of another King will at least allow a transfer that frees a downcard. 
        """
        # Check the board for a King
        
        # Snip
        
        # Check the deck for a king
        if self.deck.get_first_card() is not None and self.deck.get_first_card().value == "K":
            #king_waiting = self.deck.take_first_card() <--- BUG: CARD IS ALSO TAKEN HERE
            #print("Found a king!")
            king_loc = -1
        
        return king_loc
```

Here's what the bug looks like:

```
Move 23: Move 3 cards between piles
Move 23: Move 10 ♠ from deck to empty stack

{   'ace stacks': {'♠': '', '♡': 'A ♡', '♢': '', '♣': 'A ♣, 2 ♣'},
    'deck': '8 ♡, 6 ♡, 10 ♣, 2 ♠, 4 ♢, 3 ♡, 8 ♢, 6 ♠, 3 ♠, 7 ♣, 9 ♢, Q ♣, 7 ♠',
    'stacks': [   '10 ♠',
                  '1 cards face down., 6 ♣, 5 ♡, 4 ♠, 3 ♢',
                  '2 cards face down., Q ♠, J ♡',
                  '3 cards face down., Q ♢, J ♣, 10 ♢, 9 ♣',
                  '3 cards face down., 6 ♢, 5 ♠',
                  '5 cards face down., 9 ♡, 8 ♣',
                  '5 cards face down., 8 ♠']}
```

Now that this has been fixed, let's re-run the simulation and see what kind of results we get this time.

Strategic:

* `n=10`
    - Num wins: 3
    - Win rate: 0.3
    - Avg. num moves: 73.0
* `n=100`
    - Num wins: 14
    - Win rate: 0.14
    - Avg. num moves: 66.12
* `n=500`
    - Num wins: 115
    - Win rate: 0.23
    - Avg. num moves: 70.962
* `n=100`
    - Num wins: 182
    - Win rate: 0.182
    - Avg. num moves: 69.207

The win rate appears to have jumped by a full order of magnitude. Before, we were getting consistently less than 10%. Now, the number of wins appears to be holding steady around 20%, meaning approximately 1 in 5 games are won with the strategic algorithm playing only strategies 1, 2, 5, 6, and 7, with some of 3 thrown in there.

## Add Next Card Protection

After adding in Next Card Protection to strategy #7, I was able to maintain the win state with `seed=1`. I'm interested to test out how the win rate improves with this strategic addition, so let's run the numbers.

Strategic w/ Next Card Protection enabled:

* `n=10`
    - Num wins: 2
    - Win rate: 0.2
    - Avg. num moves: 72.8
* `n=100`
    - Num wins: 25
    - Win rate: 0.25
    - Avg. num moves: 73.82
* `n=500`
    - Num wins: 107
    - Win rate: 0.214
    - Avg. num moves: 72.304
* `n=1000`
    - Num wins: 212
    - Win rate: 0.212
    - Avg. num moves: 72.482
    
Next Card Protection on its own didn't appear to change things that much. Paired with other strategies, it may be beneficial, but without the most crucial strategy enabled, its performance gains are negligible.

**ETA:** I had only enabled NCP on cards coming from the deck, not from the board. Once I enabled them from both locations, the addition of NCP was able to get the win rate consistently above 20% for most `n`.

## Playing Smooth

This is considered THE most important strategy in the game. Now that it's been implemented, let's see if the numbers change any.

Strategic while playing smooth:

* `n=500`
    - Num wins: 86
    - Win rate: 0.172
    - Avg. num moves: 70.208
* `n=1000`
    - Num wins: 167
    - Win rate: 0.167
    - Avg. num moves: 70.057

Adding the smoothing condition actually appeared to worsen the win rate.

## Last Resorts

I've added in the final strategies, which are to play Aces wherever possible and rearrange existing piles. These functionalities were already in `take_simple_turn()`, so I've simply added the original code to `play_strategy_nine()`. With that said, here are the results.

* `n=100`
    - Num wins: 27
    - Win rate: 0.27
    - Avg. num moves: 77.78
* `n=500`
    - Num wins: 128
    - Win rate: 0.256
    - Avg. num moves: 75.726
* `n=1000`
    - Num wins: 246
    - Win rate: 0.246
    - Avg. num moves: 75.703

It appears that with all possible strategies enabled, the best we can do is about 1 in 4 odds of winning.

## Final runs: `n=1000`

After fixing any bugs and refactoring the code, here's where I got:

* Run 1:
    - Num wins: 371
    - Win rate: 0.371
    - Avg. num moves: 82.305
* Run 2:
    - Num wins: 351
    - Win rate: 0.351
    - Avg. num moves: 82.299
    
We've got it up to just a little over 1 in 3 wins, or just a little under 4 in 10 wins. According to [this website](https://gamentio.com/genblog/-/blogs/12273014), the odds for a professional solitaire player is only about 43%. We're nearly at that percentage!!

Confirmed by [this analysis](http://www.jupiterscientific.org/sciinfo/KlondikeSolitaireReport.html).

Run the code block below to generate winning stats.

In [893]:
pp = pprint.PrettyPrinter(indent = 4)

seed = None
runs = []
moves = []
turn = "strategic"
verbose = False

for i in range(1000):
    play = Game()
    play.simulate(turn=turn, verbose = verbose)

    if(play.win()):
        result = 1
    else:
        result = 0
    
    runs.append(result)
    moves.append(play.moves)
    
print(f"Num wins: {sum(runs)}\nWin rate: {sum(runs)/len(runs)}\nAvg. num moves: {np.mean(moves)}")

Num wins: 351
Win rate: 0.351
Avg. num moves: 82.299
